In [1]:
import json
import pandas as pd
import re
from google.cloud import bigquery
from google.oauth2 import service_account
import os
from datetime import datetime
from datetime import date

In [2]:
with open("koenigsegg.json", mode="r", encoding="utf-8") as f:
    data = json.load(f)
    f.close()

In [3]:
x = []
for i in data: # Loop through every page
    for j in i: # Loop through every individual car listing on a specific page
        x.append(j)

x = pd.DataFrame(x)


In [4]:
y = x.copy()
y.replace(to_replace="", value=None, inplace=True)
y["leistung"] = y["leistung"].apply(lambda x: int(re.findall(pattern="(?<=\()\d+", string=x)[0]) if x is not None else x)
y["preis"] = y["preis"].apply(lambda x: int(''.join(re.findall(pattern="\d+", string=x))) if x is not None else x)
y["kilometer"] = y["kilometer"].apply(lambda x: int(''.join(re.findall(pattern="\d+", string=x))) if x is not None else x)
y["fahrzeughalter"] = y["fahrzeughalter"].apply(lambda x: int(x) if x is not None else x)
y["standort"] = y["standort"].apply(lambda x: re.findall(pattern="[A-za-z]+(?=-)", string=x)[0] if x is not None else x)

In [5]:
y["crawled_timestamp"] = datetime.now()

In [12]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   marke                520 non-null    object        
 1   modell               520 non-null    object        
 2   variante             0 non-null      object        
 3   titel                520 non-null    object        
 4   form                 520 non-null    object        
 5   fahrzeugzustand      325 non-null    object        
 6   leistung             471 non-null    float64       
 7   getriebe             511 non-null    object        
 8   farbe                10 non-null     object        
 9   preis                520 non-null    int64         
 10  kilometer            514 non-null    float64       
 11  erstzulassung        482 non-null    object        
 12  fahrzeughalter       247 non-null    float64       
 13  standort             520 non-null  

In [7]:
y

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Koenigsegg,Beliebig,None,Koenigsegg Agera R,Sportwagen / Coupé,Unfallfrei,NaN,Automatik,None,6426000,11754,03/2014,NaN,DE,"Hallo,wir bieten einen Königsegg Agera R an.Di...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,1,2023-01-30 11:19:50.095659
1,Koenigsegg,Beliebig,None,Koenigsegg CCXR 1 of 18,Cabrio / Roadster,None,806.0,Halbautomatik,Schwarz,1429000,53500,05/2008,1.0,DE,"Wir sind stolz darauf, einen von nur 18 existi...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,1,2023-01-30 11:19:50.095659
2,Koenigsegg,Beliebig,None,Koenigsegg Andere Regera,Sportwagen / Coupé,Unfallfrei,1.0,Automatik,None,3700000,1870,10/2019,NaN,AT,"Koenigsegg Regera net price € 3.100.000,00 lik...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,1,2023-01-30 11:19:50.095659
3,Koenigsegg,Beliebig,None,Koenigsegg Agera R,Cabrio / Roadster,Unfallfrei,1.0,Automatik,Grün,3283000,3950,07/2017,1.0,GB,The Agera R made its debut at the March 2011 G...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,1,2023-01-30 11:19:50.095659
4,Koenigsegg,Beliebig,None,Koenigsegg Andere CCX,Cabrio / Roadster,None,NaN,Schaltgetriebe,Schwarz,2180000,20000,01/2006,NaN,GB,"The Koenigsegg CCX looks like its predecessor,...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,1,2023-01-30 11:19:50.095659
5,Koenigsegg,Beliebig,None,Koenigsegg CCXR EDITION // 1 OF 4,Sportwagen / Coupé,None,1.0,Schaltgetriebe,Schwarz,2889995,3028,08/2008,NaN,GB,∗FOR SALE∗ 1 OF 1 IN RHD // 1 OF 4 WORLDWIDE /...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,1,2023-01-30 11:19:50.095659
6,Koenigsegg,Beliebig,None,Koenigsegg Andere Regera 1 of 80,Sportwagen / Coupé,Unfallfrei,1.0,Automatik,Schwarz Metallic,3880000,2000,01/2020,NaN,CH,"Dear costumer, I'd like to present to you the ...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,1,2023-01-30 11:19:50.095659
7,Koenigsegg,Beliebig,None,Koenigsegg Andere Regera,Cabrio / Roadster,Unfallfrei,1.0,Automatik,Schwarz,4512000,2800,05/2021,1.0,GB,The Regera was specifically designed to be a l...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,1,2023-01-30 11:19:50.095659
8,Koenigsegg,Beliebig,None,Koenigsegg Andere JESKO 1 Slot of 125 weltweit...,"Sportwagen / Coupé, Neufahrzeug",Unfallfrei,1.0,Automatik,Weiß Metallic,4284000,50,None,NaN,DE,Wir präsentieren einen Koenigsegg JESKO 1 Slot...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,1,2023-01-30 11:19:50.095659
9,Koenigsegg,Beliebig,None,Koenigsegg Andere Regera 1 of 80 White/White C...,"Cabrio / Roadster, Neufahrzeug",Unfallfrei,1.0,Automatik,Weiß Metallic,3878805,50,None,NaN,DE,"Sehr geehrte Interessenten,Hiermit bieten wir ...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,1,2023-01-30 11:19:50.095659


In [74]:
y[y["farbe"].isnull()].loc[0, "url_to_crawl"]

'https://suchen.mobile.de/fahrzeuge/details.html?id=296584429&damageUnrepaired=ALSO_DAMAGE_UNREPAIRED&isSearchRequest=true&makeModelVariant1.makeId=13900&pageNumber=1&scopeId=C&fnai=prev&searchId=bda3cb2f-5499-bbae-6d80-24ecea55a156&ref=srp'

In [13]:
# First, set the credentials
key_path = os.getcwd() + "/bq_credentials.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Now, instantiate the client and upload the table to BigQuery
client = bigquery.Client(project="web-scraping-371310", credentials=credentials)
job_config = bigquery.LoadJobConfig(
    schema = [
        bigquery.SchemaField("marke", "STRING"),
        bigquery.SchemaField("modell", "STRING"),
        bigquery.SchemaField("variante", "STRING"),
        bigquery.SchemaField("titel", "STRING"),
        bigquery.SchemaField("form", "STRING"),
        bigquery.SchemaField("fahrzeugzustand", "STRING"),
        bigquery.SchemaField("leistung", "FLOAT64"),
        bigquery.SchemaField("getriebe", "STRING"),
        bigquery.SchemaField("farbe", "STRING"),
        bigquery.SchemaField("preis", "INT64"),
        bigquery.SchemaField("kilometer", "FLOAT64"),
        bigquery.SchemaField("erstzulassung", "STRING"),
        bigquery.SchemaField("fahrzeughalter", "FLOAT64"),
        bigquery.SchemaField("standort", "STRING"),
        bigquery.SchemaField("fahrzeugbescheibung", "STRING"),
        bigquery.SchemaField("url_to_crawl", "STRING"),
        bigquery.SchemaField("page_rank", "INT64"),
        bigquery.SchemaField("total_num_pages", "INT64"),
        bigquery.SchemaField("crawled_timestamp", "TIMESTAMP"),
    ]
)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

# Upload the table
job = client.load_table_from_dataframe(
    dataframe=y,
    destination="web-scraping-371310.crawled_datasets.lukas_mobile_de",
    job_config=job_config
).result()